# Analysis Notebook

In [ ]:
from utils import *
import os

## JG Function

In [33]:
def run_jg2(weak_answer:str, SPEC_NAME, signal_name, i, toret='string'):
    import paramiko
    import re
    from dotenv import load_dotenv
    load_dotenv()

    # SSH server details
    hostname =  os.environ.get(f"HOST_NAME")
    port = 22
    username = os.environ.get("USERNAME")
    password = os.environ.get("PASSWORD")

    # print(hostname, username, password)

    # Create SSH client
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    local_log_path = "temp.txt"

    ROOT = "/storage/ckarfa/hpdmc/"
    if SPEC_NAME == "hpdmc":
        remote_sv_file_path = ROOT + "sva/hpdmc/hpdmc.sv"
        remote_log_path = ROOT + "output.txt"
        sv_template = HPDMC_sv_template
        JG_COMMAND = """
            csh -c '
            source cad_cshrc
            cd hpdmc
            jg -batch hpdmc.tcl > output.txt
            '
        """
    elif SPEC_NAME == "i2c":
        remote_sv_file_path = ROOT + "sva/i2c/i2c.sv"
        remote_log_path = ROOT + "output.txt"
        sv_template = I2C_sv_template
        JG_COMMAND = """
            csh -c '
            source cad_cshrc
            cd hpdmc
            jg -batch i2c.tcl > output.txt
            '
        """
    def extract_svas(sv_code: str, parameter_string: str):
        # single_line_pattern = re.compile(r'assert property \(@\(.*?\) (.*?)\);', re.DOTALL)
        # Extract only valid single-line SVAs (ignoring empty property references)
        # single_line_svas = [f"assert property (@(posedge sys_clk) {sva});" for sva in single_line_pattern.findall(sv_code)]

        # Updated regex to strictly capture assertions with `@(...)`
        single_line_pattern = re.compile(r'assert property\s*\(@\([^)]*\)\s*.*?\);')
        # Extract only valid single-line SVAs while keeping their clock cycle
        single_line_svas = single_line_pattern.findall(sv_code)

        multi_line_pattern = re.compile(
            r'(property\s+\w+;.*?endproperty\s+assert property \(\w+\);)', 
            re.DOTALL
        )        
        # Extract full multi-line SVAs as a whole block
        multi_line_svas = multi_line_pattern.findall(sv_code)

        # Return only fully valid SVAs by appending with parameters
        ret = []
        for item in (single_line_svas + multi_line_svas):
            ret.append(parameter_string + item)

        return ret

    def extract_assertions(weak_answer):
        # Extract the SystemVerilog block
        code_block_matches = re.findall(r"```systemverilog(.*?)```", weak_answer, re.DOTALL)
        if not code_block_matches:
            return []
        
        code_block = code_block_matches[-1].strip()

        # Extract parameters (lines that start with "parameter")
        parameter_pattern = r"parameter\s+.*?;"
        parameters = re.findall(parameter_pattern, code_block, re.MULTILINE)
        parameter_string1 = ""
        for parameter in parameters:
            parameter_string1 += parameter + "\n"

        parameter_pattern = r"localparam\s+.*?;"
        parameters = re.findall(parameter_pattern, code_block, re.MULTILINE)
        parameter_string2 = ""
        for parameter in parameters:
            parameter_string2 += parameter + "\n"

        return extract_svas(code_block, parameter_string1 + parameter_string2)

    assertions = extract_assertions(weak_answer)

    # for item in assertions:
    #     print(item)
    #     print("-" * 80)

    def modify_template(assertion_entry):
        """Insert parameters and assertion into the SystemVerilog template."""
        return sv_template.format(properties=f"{assertion_entry}")

    def execute_ssh_commands(ssh):
        """Runs the required commands in a single SSH session."""
        command = JG_COMMAND
        stdin, stdout, stderr = ssh.exec_command(command)
        stdout.channel.recv_exit_status()  # Wait for execution

    try:
        ssh.connect(hostname, port, username, password)
        sftp = ssh.open_sftp()
        results = {}
        from time import time
        start = time()

        for assertion_entry in assertions:
            modified_sv = modify_template(assertion_entry)

            # print("Getting Feedback from JG - Starting File Upload")

            # Upload the new SystemVerilog file
            with sftp.file(remote_sv_file_path, "w") as f:
                f.write(modified_sv)

            # print("Getting Feedback from JG - File Upload Successful")

            execute_ssh_commands(ssh)

            # print("Getting Feedback from JG - Command Execution Successful. Downloading Logs")

            sftp.get(remote_log_path, local_log_path)

            with open(local_log_path, "r") as log_file:
                log_content = log_file.read()

            # Store the log output mapped to the assertion
            results[assertion_entry] = log_content

        # Close connections
        sftp.close()
        ssh.close()
        end = time()

        print("Total Time Taken to get JG Feedback: ", end - start)

        import pickle 

        with open(f'JasperGold_Logs/{SPEC_NAME}/{signal_name}_{i}.pkl', 'wb') as f:
            pickle.dump(results, f)
            
    except Exception as e:
        print(f"Error: {e}")

    # Now I only need to pass on the assertions that gave syntax error here and also check if I need to continue
    toContinue = False

    # This values are just for logging and maybe removed if needed
    CompleteCorrect = 0
    SyntaxCorrectSemanticsWrong = 0
    SyntaxWrong = 0

    for key, value in results.items():
        if "=\nSUMMARY\n=" in value:
            # Look for the "- cex                       : " pattern
            match = re.search(r"- cex\s+:\s+(\d)", value)
            
            if match:
                cex_value = int(match.group(1))  # Extract number (0 or 1)
                if cex_value == 0:
                    CompleteCorrect += 1
                else:
                    # semantically not correct according to the current RTL
                    toContinue = True
                    SyntaxCorrectSemanticsWrong += 1

            results[key] = "No Syntax Errors Found. Ensure semantics are correct wrt Specification"
        else:
            toContinue = True # Syntactically wrong assertions
            SyntaxWrong += 1

    # print(f"Results Summary:\n\tSyntax Wrong: {SyntaxWrong}\n\tSyntax Correct Semantics Wrong: {SyntaxCorrectSemanticsWrong}\n\tComplete Correct: {CompleteCorrect}")

    if toret == 'string':
        return str(results), SyntaxWrong, SyntaxCorrectSemanticsWrong, CompleteCorrect  
    elif toret == 'dict':
        return results, toContinue


## i2c Analysis

In [3]:
import pandas as pd

In [6]:
df = pd.read_csv('TempOutput/cr.csv')

In [7]:
df

,spec_name,signal_name,information,output,COMBINER
0,i2c,NaN,Here is the mapping information:\n[wb_clk_i]: ...,NaN,NaN
1,i2c,wb_clk_i,\n[Signal Name]: wb_clk_i\n[Description]:\n ...,"<think>\nOkay, let's tackle generating the SVA...","<think>\nOkay, let me tackle this problem step..."
2,i2c,wb_rst_i,\n[Signal Name]: wb_rst_i\n[Description]:\n ...,"<think>\nOkay, let's tackle generating the SVA...","<think>\nOkay, let's tackle this problem. The ..."
3,i2c,arst_i,\n[Signal name]: arst_i\n[Description]:\n [...,"<think>\nOkay, let's tackle generating the SVA...","<think>\nOkay, let's tackle this problem. The ..."
4,i2c,wb_stb_i,\n[Signal name]: wb_stb_i\n[Description]:\n ...,"<think>\nOkay, let's tackle generating the SVA...","<think>\nOkay, let me tackle this step by step..."
5,i2c,wb_ack_o,\n[Signal name]: wb_ack_o\n[Description]:\n ...,"<think>\nOkay, let's tackle generating the SVA...","<think>\nOkay, let's go through this step by s..."
6,i2c,wb_inta_o,\n[Signal name]: wb_inta_o\n[Description]:\n ...,"<think>\nOkay, let's tackle generating the SVA...","<think>\nOkay, let's tackle this problem. The ..."
7,i2c,wb_adr_i,\n[Signal name]: wb_adr_i\n[Description]:\n ...,"<think>\nOkay, let's tackle generating the SVA...","<think>\nOkay, let's see. The user wants me to..."
8,i2c,wb_dat_i,\n[Signal name]: wb_dat_i\n[Description]:\n ...,"<think>\nOkay, let's tackle generating the SVA...","<think>\nOkay, let me try to tackle this probl..."
9,i2c,wb_cyc_i,\n[Signal name]: wb_cyc_i\n[Description]:\n ...,"<think>\nOkay, let's tackle generating the SVA...","<think>\nOkay, let's start by looking at the u..."


In [37]:
from tabulate import tabulate

table_data = [
    ["Signal Name", "Syntax Wrong", "Syntax Wrong, Semantics Correct", "Complete Correct"]
]

# Populate the rows with data
for index, row in df.iterrows():
    if index == 0:
        continue

    with open(f"{row['signal_name']}.txt", 'r') as f:
        text = f.read()

    _, a, b, c = run_jg2(text, row['spec_name'], row['signal_name'], 'Analysis')
    print(f"{[row['signal_name'], a, b, c]}")
    
    table_data.append([row['signal_name'], a, b, c])

Total Time Taken to get JG Feedback:  86.40209794044495
['wb_clk_i', 0, 4, 5]
Total Time Taken to get JG Feedback:  199.75174379348755
['wb_rst_i', 0, 0, 17]
Total Time Taken to get JG Feedback:  489.4732208251953
['arst_i', 1, 1, 13]
Total Time Taken to get JG Feedback:  39.0348904132843
['wb_stb_i', 0, 5, 1]
Total Time Taken to get JG Feedback:  152.30940198898315
['wb_ack_o', 0, 7, 9]
Total Time Taken to get JG Feedback:  155.2459819316864
['wb_inta_o', 0, 4, 11]
Total Time Taken to get JG Feedback:  45.226327657699585
['wb_adr_i', 0, 6, 1]
Total Time Taken to get JG Feedback:  104.2190477848053
['wb_dat_i', 0, 9, 4]
Total Time Taken to get JG Feedback:  89.7972023487091
['wb_cyc_i', 0, 11, 2]
Total Time Taken to get JG Feedback:  189.96967363357544
['wb_dat_o', 0, 17, 3]
Total Time Taken to get JG Feedback:  88.5073037147522
['wb_we_i', 0, 6, 4]
Total Time Taken to get JG Feedback:  29.27069091796875
['scl_pad_i', 0, 3, 1]
Total Time Taken to get JG Feedback:  62.83789873123169
['s

In [38]:
# Print the formatted table
print("Analysis Table after Automated Combiner for i2c Specification:")
print(tabulate(table_data, headers="firstrow", tablefmt="grid"))

Analysis Table after Automated Combiner for i2c Specification:
+---------------+----------------+-----------------------------------+--------------------+
| Signal Name   |   Syntax Wrong |   Syntax Wrong, Semantics Correct |   Complete Correct |
+===============+================+===================================+====================+
| wb_clk_i      |              0 |                                 4 |                  5 |
+---------------+----------------+-----------------------------------+--------------------+
| wb_rst_i      |              0 |                                 0 |                 17 |
+---------------+----------------+-----------------------------------+--------------------+
| arst_i        |              1 |                                 1 |                 13 |
+---------------+----------------+-----------------------------------+--------------------+
| wb_stb_i      |              0 |                                 5 |                  1 |
+---------------+

In [41]:
table_data

[['Signal Name',
  'Syntax Wrong',
  'Syntax Wrong, Semantics Correct',
  'Complete Correct'],
 ['wb_clk_i', 0, 4, 5],
 ['wb_rst_i', 0, 0, 17],
 ['arst_i', 1, 1, 13],
 ['wb_stb_i', 0, 5, 1],
 ['wb_ack_o', 0, 7, 9],
 ['wb_inta_o', 0, 4, 11],
 ['wb_adr_i', 0, 6, 1],
 ['wb_dat_i', 0, 9, 4],
 ['wb_cyc_i', 0, 11, 2],
 ['wb_dat_o', 0, 17, 3],
 ['wb_we_i', 0, 6, 4],
 ['scl_pad_i', 0, 3, 1],
 ['scl_pad_o', 0, 0, 6],
 ['sda_pad_i', 0, 3, 5],
 ['sda_pad_o', 0, 0, 4],
 ['scl_pad_oe', 0, 9, 5],
 ['sda_pad_oe', 0, 2, 18],
 ['ctr', 0, 4, 7],
 ['sr', 4, 9, 12],
 ['prer', 0, 5, 5],
 ['txr', 1, 3, 10],
 ['rxr', 0, 8, 5],
 ['cr', 0, 13, 4]]

In [42]:
df = pd.DataFrame(table_data[1:], columns=table_data[0])
print(df)

   Signal Name  Syntax Wrong  Syntax Wrong, Semantics Correct  \
0     wb_clk_i             0                                4   
1     wb_rst_i             0                                0   
2       arst_i             1                                1   
3     wb_stb_i             0                                5   
4     wb_ack_o             0                                7   
5    wb_inta_o             0                                4   
6     wb_adr_i             0                                6   
7     wb_dat_i             0                                9   
8     wb_cyc_i             0                               11   
9     wb_dat_o             0                               17   
10     wb_we_i             0                                6   
11   scl_pad_i             0                                3   
12   scl_pad_o             0                                0   
13   sda_pad_i             0                                3   
14   sda_pad_o           

In [43]:
df.to_csv('i2cAnalysis.csv', index=False)